#DATA READING 
using pyspark api

In [0]:
df1 = spark.read.format('parquet')\
    .option('header','True')\
        .option('inferSchema',True)\
        .load('abfss://bronze@carkrishdatalake.dfs.core.windows.net/Rawdata')

In [0]:
df1.display()

##DATA TRANSFORMATION

In [0]:
from pyspark.sql.functions  import *;

In [0]:
df1 =df1.withColumn('Model_Catergory',split(col('Model_ID'),'-')[0]);

###cast 
is used as typecast function from one datatype to anotherdatatype 

In [0]:
df1.withColumn('Units_Sold',col('Units_Sold').cast(StringType())).printSchema()

###Arthematic_operations 
between Revenue and UnitsSold

In [0]:
df1 = df1.withColumn('RevenuePerUnits',col('Revenue')/col('Units_Sold'));

# AD-HOC

In [0]:
df1.display()

In [0]:
display(df1.groupBy('Year','BranchName').agg(sum('Units_Sold').alias ('Total_Units')).sort('Year','Total_Units',ascending=[1,0,0]))

###Data Writing

In [0]:
df1.write.format('parquet')\
        .mode('append')\
          .option('path','abfss://silver@carkrishdatalake.dfs.core.windows.net/Carsales')\
            .save()

##Querying Silver Data

In [0]:
from pyspark.sql import SparkSession

In [0]:
%sql
select * from parquet.`abfss://silver@carkrishdatalake.dfs.core.windows.net/Carsales`;